# Demo 3 — Hierarchical Decision-Flow Simulation
## Multi-Echelon Decision Decomposition with Hybrid Reasoning

**OODA Phase: Decide / Act (across echelons)**

**Purpose:** Run a compact micro-wargame where decisions decompose across a three-level hierarchy: a Commander sets intent and prioritizes objectives, Managers allocate resources and create synchronized sub-plans, and Operators execute actions on a simple grid. A Multi-Model Selector visibly chooses between scripted heuristics and LLM reasoning at each decision step, demonstrating the hybrid multi-model concept.

**Audience:** Warfighters, operational planners, and C2 staff familiar with hierarchical command structures.
**Primary outcome:** The audience sees how multi-agent orchestration mirrors staff-like structures — Commander → Warfare Managers → Unit Operators — with visible reasoning selection between heuristic and LLM at each decision point.

---

> ### Responsible AI & Scope Statement
> *This research explores human-AI collaboration and explainable decision-support in fully synthetic, abstract environments. All scenarios, agents, and data are artificially generated. No real-world operational data, contingency plans, systems, or intelligence are used or represented. The prototype is intended exclusively for research, educational, and experimental purposes and does not constitute an operational model, validated planning tool, or source of decision authority. All agent outputs are recommendations and analysis — not orders or directives. Final decisions remain with the human operator at all times.*

## What It Illustrates (Multi-Agent)

| Agent | Role | Echelon |
|-------|------|---------|
| **Commander** | Sets strategic intent, decomposes objectives, recommends risk tolerance | Strategic |
| **Surface Warfare Manager** | Proposes surface unit positions, creates synchronized sub-plan, coordinates with Air Mgr | Operational |
| **Air Warfare Manager** | Proposes air/ISR asset positions, creates synchronized sub-plan, coordinates with Surface Mgr | Operational |
| **Operator Agents (×4)** | Carry out recommended tactical actions on grid: patrol, screen, intercept (with HITL gate), conserve | Tactical |
| **Multi-Model Selector** | Routes each decision to scripted heuristic or LLM based on complexity/novelty | Cross-cutting |
| **Feedback Aggregator** | Collects outcomes, surfaces results upward, enables mid-game adaptation | Cross-cutting |
| **Explainability Agent** | Produces transparent reasoning traces explaining why each agent reached its conclusion | Cross-cutting |

**Success criteria:** Over 3 turns the Commander's recommended intent flows down through Managers to Operators, actions resolve on a visible grid, the Multi-Model Selector log shows when/why heuristics vs. LLM reasoning were chosen, feedback flows back up to inform adaptation, and the Explainability Agent provides an auditable trace of each decision chain.


## Demo Script (Presenter Guide)

1. **Intro (1 min):** "Every warfighter works in a hierarchy. This demo shows how AI agents can mirror Commander–Manager–Operator structures — each reasoning at the right level of abstraction. All scenario content is fully synthetic and fictional."
2. **Config (30 sec):** Point out the grid, force laydown, and scenario selection. "The Commander sees the strategic picture; Operators see only their local area. All unit names and locations are invented for this exercise."
3. **Turn 1 (2 min):** Run and narrate: "Watch the Commander recommend intent, Managers propose force allocations, and Operators carry out actions. The sidebar shows which decisions used heuristics vs. LLM reasoning."
4. **Turns 2–3 (3 min):** "The situation changes. Notice how feedback from Operators flows back up and the Commander adapts recommendations mid-game. Any intercept actions are flagged for human confirmation."
5. **Multi-Model Selector (1 min):** "Point out the decision log — routine patrol uses cheap heuristics, novel contacts trigger LLM reasoning. Same architecture, swappable engines."
6. **Explainability (1 min):** "Review the Explainability Agent's reasoning trace — it shows WHY each agent reached its conclusion, making the full decision chain auditable."
7. **Close (1 min):** "This is the Decide/Act loop across echelons — with every decision traceable, explainable, and subject to human oversight. The agents recommend, they do not direct."


## Azure Technologies Used in This Demo

This demo relies on several Azure services and open-source frameworks. Here is a brief overview for those new to Azure:

| Technology | What It Does | Learn More |
|------------|-------------|------------|
| **Azure AI Foundry** | A unified platform for building, evaluating, and deploying AI models. It provides a model catalog, prompt engineering tools, and managed endpoints so you can host and call LLMs without managing infrastructure. | [Azure AI Foundry documentation](https://learn.microsoft.com/azure/ai-studio/what-is-ai-studio) |
| **Azure AI Model Inference API** | A common REST API that lets you call any model deployed in Azure AI Foundry with a single, consistent interface — regardless of the underlying model provider (OpenAI, Mistral, Meta, etc.). | [Azure AI Model Inference API](https://learn.microsoft.com/azure/ai-studio/reference/reference-model-inference-api) |
| **Azure Key Vault** | A cloud service for securely storing and managing secrets (API keys, certificates, connection strings). This project uses Key Vault to keep Azure AI credentials out of source code. | [Azure Key Vault overview](https://learn.microsoft.com/azure/key-vault/general/overview) |
| **AutoGen** | A Microsoft open-source framework for building multi-agent AI applications. Agents are defined with system prompts and model clients, then orchestrated in teams (e.g., `RoundRobinGroupChat`). | [AutoGen documentation](https://microsoft.github.io/autogen/) |
| **`autogen-ext[azure]`** | The Azure extension for AutoGen, providing `AzureAIChatCompletionClient` — a ready-made model client that connects AutoGen agents to Azure AI Foundry endpoints. | [AutoGen Azure extension](https://microsoft.github.io/autogen/stable/reference/python/autogen_ext.models.azure.html) |
| **Azure Identity SDK** | The `azure-identity` Python package provides credential classes (like `DefaultAzureCredential`) for authenticating to Azure services without hard-coding secrets. | [Azure Identity client library](https://learn.microsoft.com/python/api/overview/azure/identity-readme) |

> **Tip:** If you are completely new to Azure, start with the [Azure Fundamentals learning path](https://learn.microsoft.com/training/paths/az-900-describe-cloud-concepts/) which covers core cloud concepts, Azure services, and pricing.

## Setup

Run once per environment. This demo requires the following Python packages:

- **`autogen-agentchat`** — Core [AutoGen](https://microsoft.github.io/autogen/) library for creating and orchestrating AI agents.
- **`autogen-ext[azure]`** — Azure extension that provides `AzureAIChatCompletionClient`, connecting AutoGen agents to models deployed in [Azure AI Foundry](https://learn.microsoft.com/azure/ai-studio/what-is-ai-studio).
- **`python-dotenv`** — Loads environment variables from a `.env` file so you don't hard-code secrets.

### Required Environment Variables

| Variable | Purpose |
|----------|---------|
| `AZURE_INFERENCE_ENDPOINT` | The URL of your model deployment in Azure AI Foundry (e.g., `https://<your-resource>.services.ai.azure.com/models`). You can find this on the **Deployments** page in [Azure AI Foundry portal](https://ai.azure.com). |
| `AZURE_INFERENCE_CREDENTIAL` | An API key that authenticates requests to your Azure AI endpoint. Retrieve it from your Azure AI resource's **Keys and Endpoint** page, or store it in [Azure Key Vault](https://learn.microsoft.com/azure/key-vault/general/overview) for production use. |

> **First time?** See [Deploy models with Azure AI Foundry](https://learn.microsoft.com/azure/ai-studio/how-to/deploy-models) for a step-by-step walkthrough of creating an endpoint and obtaining credentials.

In [1]:
# Uncomment to install dependencies
# %pip install -U "autogen-agentchat==0.7.5" "autogen-ext[azure]==0.7.5" python-dotenv

In [ ]:
# ═══════════════════════════════════════════════════════════════
# NAML 2026 BOOTSTRAP v2 — Survives dead AML mounts (Errno 107)
# ═══════════════════════════════════════════════════════════════

import os
import sys

def _safe_stat(path: str) -> bool:
    try:
        os.stat(path)
        return True
    except OSError:
        return False

def _prune_dead_sys_path():
    kept = []
    removed = []
    for p in list(sys.path):
        if not p:
            kept.append(p)
            continue
        if _safe_stat(p):
            kept.append(p)
        else:
            removed.append(p)
    sys.path[:] = kept
    print(f"✓ Pruned sys.path. Removed {len(removed)} dead entries.")
    return removed

def _safe_listdir(path: str):
    try:
        return os.listdir(path)
    except OSError:
        return None

def _find_repo_root(marker_dir: str = "common", start_candidates=None, max_up: int = 6):
    """
    Find a repo root by looking for a marker directory (e.g., 'common').
    Avoids Path.exists()/stat on dead mounts by only using listdir on traversable dirs.
    """
    if start_candidates is None:
        start_candidates = []

    # Candidate starting points:
    #  - current working directory (may be dead)
    #  - directory of the notebook file if available via env (sometimes set)
    #  - user home (often stable)
    candidates = [os.getcwd()] + start_candidates + [os.path.expanduser("~")]

    checked = set()
    for base in candidates:
        cur = base
        for _ in range(max_up + 1):
            if cur in checked:
                break
            checked.add(cur)

            entries = _safe_listdir(cur)
            if entries is not None and marker_dir in entries:
                return cur  # found repo root

            parent = os.path.dirname(cur)
            if parent == cur:
                break
            cur = parent

    return None

# 1) prune dead sys.path entries
_prune_dead_sys_path()

# 2) find a safe repo root by locating the 'common/' folder
repo_root = _find_repo_root(marker_dir="common", start_candidates=[])

if repo_root:
    sys.path.insert(0, repo_root)
    print(f"✓ Repo root added: {repo_root}")
else:
    print("✗ Could not find repo root safely (mount may be disconnected).")
    print("  Fix: restart kernel/compute, or run from a local (non-/mnt) working copy.")

print("✓ Bootstrap complete.")


## Imports

The cell below loads the libraries this demo depends on. Here are the key Azure and AutoGen imports explained:

| Import | What It Provides |
|--------|-----------------|
| `AzureAIChatCompletionClient` | From `autogen-ext[azure]`. A model client that speaks the [Azure AI Model Inference API](https://learn.microsoft.com/azure/ai-studio/reference/reference-model-inference-api), allowing AutoGen agents to call any model deployed in Azure AI Foundry. |
| `AzureKeyCredential` | From the [Azure Core SDK](https://learn.microsoft.com/python/api/azure-core/azure.core.credentials.azurekeycredential). Wraps an API key string into a credential object that Azure client libraries expect. |
| `AssistantAgent` | An AutoGen agent backed by an LLM. You give it a system prompt and a model client; it generates responses via the connected Azure endpoint. See [AutoGen AssistantAgent](https://microsoft.github.io/autogen/stable/reference/python/autogen_agentchat.agents.html). |
| `RoundRobinGroupChat` | An AutoGen team pattern where agents take turns speaking in a fixed order — used here to simulate the C2 hierarchy (Commander → Managers → Operators → Feedback → Explainability). |
| `ModelFamily` | Metadata enum describing model capabilities (vision, function calling, JSON output). Helps AutoGen adapt its prompting strategy to the deployed model. |

In [ ]:
# ═══════════════════════════════════════════════════════════════
# NAML 2026 BOOTSTRAP — Safe Import Setup for Azure ML
# ═══════════════════════════════════════════════════════════════

import os
import sys
from pathlib import Path

def _prune_dead_sys_path():
    """Remove broken mount paths (common in Azure ML after remount issues)."""
    cleaned = []
    for p in sys.path:
        try:
            if not p or os.path.exists(p):
                cleaned.append(p)
        except OSError:
            # Dead mount or broken transport endpoint
            pass
    sys.path[:] = cleaned

def _add_repo_root(levels_up: int = 2):
    """
    Add repo root relative to notebook location.
    Adjust `levels_up` if folder depth changes.
    """
    repo_root = Path(os.getcwd()).resolve()
    for _ in range(levels_up):
        repo_root = repo_root.parent

    if repo_root.exists():
        sys.path.insert(0, str(repo_root))
        print(f"✓ Repo root added: {repo_root}")
    else:
        print("⚠ Repo root not found:", repo_root)

# Execute bootstrap steps
_prune_dead_sys_path()
_add_repo_root(levels_up=2)

print("✓ Bootstrap complete.")


In [2]:
import json
import os
import random
import sys
from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Tuple

from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.conditions import MaxMessageTermination
from autogen_agentchat.base import TaskResult
from autogen_core.models import ModelFamily
from autogen_ext.models.azure import AzureAIChatCompletionClient
from azure.core.credentials import AzureKeyCredential
from IPython.display import display, HTML, Markdown

# Optional: load .env for API keys
try:
    from dotenv import load_dotenv
    load_dotenv()
except ImportError:
    pass

# ── Project-common utilities ───────────────────────────────────
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), "..", "..")))

from common.config import (
    Side, UnitType, Terrain, GRID_SIZE, NUM_WARGAME_TURNS,
    DEFAULT_MODEL,
    ENV_AZURE_INFERENCE_ENDPOINT, ENV_AZURE_INFERENCE_CREDENTIAL,
 )
from common.ui import (
    render_grid_html as _render_grid_html_common,
    render_turn_header, render_escalation_banner, render_hr,
    render_selector_summary, render_summary_card,
 )
from common.logging import (
    log_info, log_success, log_warning, log_error,
    log_step, log_metric, clear_logs,
 )

log_success("All imports loaded.")

LogEntry(level='SUCCESS', message='All imports loaded.', timestamp='2026-02-12 19:15:00', extra={})

## LLM Configuration

This cell creates the model client that every agent in the demo will use to generate responses.

### How It Works

1. **Environment variables** (`AZURE_INFERENCE_ENDPOINT` and `AZURE_INFERENCE_CREDENTIAL`) point to a model deployed in [Azure AI Foundry](https://learn.microsoft.com/azure/ai-studio/what-is-ai-studio).
2. **`AzureAIChatCompletionClient`** connects AutoGen to that endpoint using the [Azure AI Model Inference API](https://learn.microsoft.com/azure/ai-studio/reference/reference-model-inference-api). This is the same API regardless of which model you deploy (GPT-4o, Mistral, Llama, etc.), so you can swap models without changing code.
3. **`model_info`** describes the model's capabilities — AutoGen uses this metadata to decide how to format prompts (e.g., whether to use structured JSON output, function calling, or plain text).
4. **`AzureKeyCredential`** wraps your API key into a credential object. For production workloads, consider using [managed identity](https://learn.microsoft.com/entra/identity/managed-identities-azure-resources/overview) with `DefaultAzureCredential` instead of API keys.

> **Swap models easily:** Because the Azure AI Model Inference API is model-agnostic, you can change `FOUNDRY_MODEL` to any model deployed in your Azure AI Foundry project — no other code changes needed. See [Supported models in Azure AI Foundry](https://learn.microsoft.com/azure/ai-studio/how-to/model-catalog-overview) for a full catalog.

In [3]:
# ── LLM Configuration ──────────────────────────────────────────
# Azure AI Foundry / Azure AI Inference → set AZURE_INFERENCE_ENDPOINT + AZURE_INFERENCE_CREDENTIAL

# Hard-code the model ID you want to use for this demo.
FOUNDRY_MODEL = DEFAULT_MODEL

def build_model_client():
    """Build an AutoGen 0.7 model client for Azure AI Foundry models."""
    missing = [
        name for name in (ENV_AZURE_INFERENCE_ENDPOINT, ENV_AZURE_INFERENCE_CREDENTIAL)
        if not os.environ.get(name)
    ]
    if missing:
        raise EnvironmentError(
            "Missing Azure AI Foundry / Inference configuration. Set:\n"
            f"  {ENV_AZURE_INFERENCE_ENDPOINT}\n"
            f"  {ENV_AZURE_INFERENCE_CREDENTIAL}\n"
        )

    model_info = {
        "family": ModelFamily.UNKNOWN,
        "vision": False,
        "function_calling": False,
        "json_output": True,
        "structured_output": False,
        "multiple_system_messages": True,
    }

    return AzureAIChatCompletionClient(
        endpoint=os.environ[ENV_AZURE_INFERENCE_ENDPOINT],
        credential=AzureKeyCredential(os.environ[ENV_AZURE_INFERENCE_CREDENTIAL]),
        model=FOUNDRY_MODEL,
        model_info=model_info,
    )

model_client = build_model_client()
log_success(f"Model client: AzureAIChatCompletionClient (Foundry) — {FOUNDRY_MODEL}")

LogEntry(level='SUCCESS', message='Model client: AzureAIChatCompletionClient (Foundry) — gpt-4o', timestamp='2026-02-12 19:15:00', extra={})

## Battlespace Grid, Forces & Scenario Data

A **10×10 text-based grid** representing the fictional Cerulean Strait chokepoint. BLUE forces must control the strait while preserving force. RED forces are contesting passage. Each cell has terrain type and can hold units. Operators see only their assigned 3×3 local area; Managers see their warfare domain; the Commander sees the entire grid at reduced resolution.

> *All locations, unit names, and scenario events are entirely synthetic and fictional. No real-world geography, forces, or operational data are represented.*

In [ ]:
# ── Grid & Terrain ─────────────────────────────────────────────
# Side, UnitType, Terrain, GRID_SIZE imported from common.config

# Row 4-5 is the strait chokepoint
TERRAIN_MAP: List[List[str]] = []
for r in range(GRID_SIZE):
    row = []
    for c in range(GRID_SIZE):
        if r in (4, 5):
            row.append(Terrain.STRAIT.value)
        elif (r, c) in [(3, 2), (3, 3), (6, 7), (6, 8)]:
            row.append(Terrain.ISLAND.value)
        elif (r, c) == (0, 0):
            row.append(Terrain.PORT.value)
        elif (r, c) == (9, 9):
            row.append(Terrain.PORT.value)
        elif r in (3, 6) and c not in (2, 3, 7, 8):
            row.append(Terrain.SHALLOWS.value)
        else:
            row.append(Terrain.OPEN_WATER.value)
    TERRAIN_MAP.append(row)


# ── Unit Dataclass (demo-specific) ─────────────────────────────

@dataclass
class Unit:
    name: str
    unit_type: UnitType
    side: Side
    position: Tuple[int, int]
    status: str = "ready"
    mission: str = "none"
    warfare_domain: str = "surface"  # surface or air
    health: int = 100

    def symbol(self) -> str:
        side_char = "B" if self.side == Side.BLUE else "R"
        return f"{side_char}{self.unit_type.value[0]}"

# ── Force Laydown ──────────────────────────────────────────────

blue_forces: List[Unit] = [
    Unit("BLUE Destroyer Alpha", UnitType.DDG, Side.BLUE, (2, 1), warfare_domain="surface",
         mission="patrol"),
    Unit("BLUE Frigate Bravo", UnitType.FFG, Side.BLUE, (3, 5), warfare_domain="surface",
         mission="strait screening"),
    Unit("BLUE Patrol Craft Charlie", UnitType.PATROL, Side.BLUE, (5, 3), warfare_domain="surface",
         mission="close patrol"),
    Unit("BLUE MPA Delta", UnitType.MPA, Side.BLUE, (1, 4), warfare_domain="air",
         mission="maritime ISR"),
    Unit("BLUE UAV Echo", UnitType.UAV, Side.BLUE, (4, 7), warfare_domain="air",
         mission="surface search"),
]

red_forces: List[Unit] = [
    Unit("RED Frigate Foxtrot", UnitType.FFG, Side.RED, (7, 6), warfare_domain="surface",
         mission="area denial"),
    Unit("RED Patrol Craft Golf", UnitType.PATROL, Side.RED, (8, 4), warfare_domain="surface",
         mission="intercept"),
    Unit("RED Submarine Hotel", UnitType.SUB, Side.RED, (6, 2), warfare_domain="surface",
         mission="lurk"),
]

all_units = blue_forces + red_forces

# ── Scenario Injections (per turn) ─────────────────────────────

TURN_EVENTS: List[Dict[str, Any]] = [
    {   # Turn 1: baseline
        "description": "Routine patrol — no unexpected contacts.",
        "red_moves": {},             # no RED movement
        "new_contacts": [],
        "comms_degraded": False,
        "novel_encounter": False,
    },
    {   # Turn 2: RED repositions, new contact
        "description": "RED forces repositioning south. New subsurface contact detected.",
        "red_moves": {"RED Frigate Foxtrot": (6, 5), "RED Patrol Craft Golf": (7, 3)},
        "new_contacts": [
            {"name": "Unknown Sub Contact", "type": UnitType.SUB, "position": (5, 8),
             "side": Side.RED, "confidence": "LOW"},
        ],
        "comms_degraded": False,
        "novel_encounter": True,
    },
    {   # Turn 3: escalation — comms degraded, aggressive RED
        "description": "Electronic warfare degrades comms. RED frigate closing the strait.",
        "red_moves": {"RED Frigate Foxtrot": (5, 5), "RED Patrol Craft Golf": (6, 3),
                      "RED Submarine Hotel": (5, 2)},
        "new_contacts": [],
        "comms_degraded": True,
        "novel_encounter": True,
    },
]


# ── Grid Rendering (adapters around common.ui) ────────────────

def _build_unit_positions(units: List[Unit]) -> Dict[Tuple[int, int], Unit]:
    """Build {(row, col): unit} dict for the common render_grid_html."""
    return {u.position: u for u in units}


def render_grid(units: List[Unit], *, highlight_area: Optional[Tuple[Tuple[int,int],Tuple[int,int]]] = None) -> str:
    """Render the grid as text. Units overlay terrain symbols."""
    grid = [row[:] for row in TERRAIN_MAP]  # deep copy
    for u in units:
        r, c = u.position
        if 0 <= r < GRID_SIZE and 0 <= c < GRID_SIZE:
            grid[r][c] = u.symbol()
    header = "   " + " ".join(f"{c:>3}" for c in range(GRID_SIZE))
    lines = [header]
    for r in range(GRID_SIZE):
        row_str = " ".join(f"{grid[r][c]:>3}" for c in range(GRID_SIZE))
        lines.append(f"{r:>2} {row_str}")
    return "\n".join(lines)


def render_grid_html(units: List[Unit]) -> str:
    """Render the grid as HTML using shared common.ui helper."""
    return _render_grid_html_common(
        terrain_map=TERRAIN_MAP,
        unit_positions=_build_unit_positions(units),
        grid_size=GRID_SIZE,
    )


# ── Runtime State ──────────────────────────────────────────────
turn_history: List[Dict[str, Any]] = []
selector_log: List[Dict[str, str]] = []    # Multi-Model Selector decisions
current_turn: int = 0

log_metric("Grid size", f"{GRID_SIZE}×{GRID_SIZE}")
log_metric("BLUE units", len(blue_forces))
log_metric("RED units", len(red_forces))
log_metric("Turns available", NUM_WARGAME_TURNS)
log_info("Initial grid:")
display(HTML(render_grid_html(all_units)))

,0,1,2,3,4,5,6,7,8,9
0,P,~,~,~,~,~,~,~,~,~
1,~,~,~,~,BM,~,~,~,~,~
2,~,BD,~,~,~,~,~,~,~,~
3,.,.,#,#,.,BF,.,~,~,.
4,=,=,=,=,=,=,=,BU,=,=
5,=,=,=,BP,=,=,=,=,=,=
6,.,.,RS,~,.,.,.,#,#,.
7,~,~,~,~,~,~,RF,~,~,~
8,~,~,~,~,RP,~,~,~,~,~
9,~,~,~,~,~,~,~,~,~,P


## Agent Definitions — Hierarchical C2 Team

Seven [AutoGen `AssistantAgent`](https://microsoft.github.io/autogen/stable/reference/python/autogen_agentchat.agents.html) instances arranged in a three-level hierarchy. Each agent is constructed with two key inputs:

- **`system_message`** — A detailed prompt that scopes the agent's echelon of responsibility, information visibility, and output format. This is sent to the LLM as the system/developer message on every call.
- **`model_client`** — The `AzureAIChatCompletionClient` created above, which routes all LLM calls to [Azure AI Foundry](https://learn.microsoft.com/azure/ai-studio/what-is-ai-studio).

Every agent produces **recommendations**, not directives — final authority remains with the human operator. The Multi-Model Selector, Feedback Aggregator, and Explainability Agent cut across levels.

> **AutoGen concept:** Each `AssistantAgent` is a stateful wrapper around an LLM call. When the agent "speaks" inside a team, AutoGen sends the conversation history plus the agent's system prompt to the Azure AI endpoint, receives the response, and appends it to the shared chat log. See [Building agents in AutoGen](https://microsoft.github.io/autogen/stable/user-guide/agentchat-user-guide/quickstart.html) for an introduction.

In [ ]:
# ── Agent System Prompts ───────────────────────────────────────

COMMANDER_PROMPT = """\
You are the **Fleet Commander** in a fully synthetic naval micro-wargame exercise.
All decisions remain with the human operator. You provide analysis and options, not directives.

ECHELON: Strategic. You see the ENTIRE grid at reduced detail.

RESPONSIBILITIES:
1. Receive the high-level objective and current battlespace picture.
2. Decompose the objective into 2-3 prioritized sub-objectives.
3. Recommend each sub-objective to a Warfare Manager (Surface or Air) with:
   - A suggested operating area (grid row/column ranges)
   - Risk tolerance (CAUTIOUS / BALANCED / AGGRESSIVE)
   - Priority ranking
4. Adapt recommendations mid-game when feedback reveals the situation has changed.

RULES:
- Never recommend individual unit actions — that is the Managers' responsibility.
- Provide intent statements and recommendations, NOT detailed orders.
- If feedback shows high casualties or mission failure, recommend shifting priorities.
- Acknowledge uncertainty and state what you need to know.

OUTPUT FORMAT (use EXACTLY):
**COMMANDER'S INTENT (RECOMMENDATION) — Turn [N]**

OBJECTIVE: [one sentence]

SUB-OBJECTIVES (prioritized):
1. [Sub-obj] → Recommended for: [Surface/Air Warfare Manager]
   Suggested Operating Area: rows [X-Y], cols [X-Y]
   Risk Tolerance: [CAUTIOUS/BALANCED/AGGRESSIVE]
2. [Sub-obj] → Recommended for: [Manager]
   ...

GUIDANCE: [1-2 sentences of commander's assessment / adaptation from feedback]

DECISION POINTS AHEAD: [what would trigger a change in recommendations]"""


You are the **Surface Warfare Manager** in a fully synthetic naval micro-wargame exercise.
All decisions remain with the human operator. You provide analysis and options, not directives.

ECHELON: Operational. You see your suggested operating area on the grid.

ASSIGNED ASSETS: DDG, FFG, Patrol Craft (surface units only).

RESPONSIBILITIES:
1. Receive the Commander's recommended intent and your suggested sub-objective.
2. Propose positions and missions for your surface units within your operating area.
3. Coordinate with the Air Warfare Manager when you need ISR coverage or air support.
4. Identify resource conflicts and propose synchronized timing.
5. Provide recommended actions for each Operator under your coordination.

OUTPUT FORMAT:
**SURFACE WARFARE PLAN (RECOMMENDATION) — Turn [N]**

RECEIVED INTENT: [summarize commander's sub-objective for you]

UNIT RECOMMENDATIONS:
• [Unit name] → Suggested Action: [PATROL/SCREEN/INTERCEPT/CONSERVE] at ([row],[col])
  Rationale: [one sentence]

COORDINATION REQUEST TO AIR WARFARE:
• [request, if any, or "None"]

RISK ASSESSMENT: [one sentence about the main risk in this plan]"""


You are the **Air Warfare Manager** in a fully synthetic naval micro-wargame exercise.
All decisions remain with the human operator. You provide analysis and options, not directives.

ECHELON: Operational. You see your suggested operating area on the grid.

ASSIGNED ASSETS: MPA, UAV — air and ISR assets only.

RESPONSIBILITIES:
1. Receive the Commander's recommended intent and your suggested sub-objective.
2. Propose air/ISR asset positions and missions within your operating area.
3. Respond to coordination requests from the Surface Warfare Manager.
4. Prioritize ISR collection to fill information gaps.
5. Provide recommended actions for each Operator under your coordination.

OUTPUT FORMAT:
**AIR WARFARE PLAN (RECOMMENDATION) — Turn [N]**

RECEIVED INTENT: [summarize commander's sub-objective for you]

UNIT RECOMMENDATIONS:
• [Unit name] → Suggested Action: [PATROL/SCREEN/INTERCEPT/CONSERVE] at ([row],[col])
  Rationale: [one sentence]

COORDINATION RESPONSE TO SURFACE WARFARE:
• [response to any request, or "Standing by"]

ISR PRIORITIES:
1. [Sector/target to search]
2. [Secondary]"""


OPERATOR_PROMPT = """\
You are a **Unit Operator** in a fully synthetic naval micro-wargame exercise.
All decisions remain with the human operator. You provide analysis and options, not directives.

ECHELON: Tactical. You see ONLY your local 3×3 area on the grid.

ACTIONS AVAILABLE:
- PATROL: Move to an adjacent cell and observe.
- SCREEN: Hold position and provide early warning across your 3×3 area.
- INTERCEPT: Move toward a contact and recommend engagement (requires human confirmation before proceeding).
- CONSERVE: Minimize exposure, reduce signature, preserve readiness.

RESPONSIBILITIES:
1. Receive the recommended action from your Warfare Manager.
2. Report what you observe in your local area (any contacts, terrain).
3. Carry out the recommended action OR request clarification if the recommendation conflicts
   with what you see locally (e.g., recommended to intercept but no contacts visible).
4. For INTERCEPT actions: flag that human confirmation is required before engagement.
5. Report outcome: success, partial, or pending-confirmation with explanation.

OUTPUT FORMAT:
**OPERATOR REPORT — [Unit Name] — Turn [N]**
Position: ([row],[col])
Local contacts: [list or "None"]
Recommendation received: [action] at ([target row],[col])
Execution: [CARRIED OUT / MODIFIED — reason / AWAITING HUMAN CONFIRMATION] → Result: [outcome]
Readiness: [health]%"""


MULTI_MODEL_SELECTOR_PROMPT = """\
You are the **Multi-Model Selector** — a lightweight coordinator that decides
HOW each decision should be made in the wargame.

For each decision point presented to you, determine:
1. Is this ROUTINE (predictable, well-covered by doctrine) → use HEURISTIC
2. Is this NOVEL or COMPLEX (unexpected contacts, conflicting orders, ambiguity) → use LLM

CRITERIA FOR HEURISTIC:
- Standard patrol movements with no contacts
- Screening in assigned area with clear orders
- Conservation when no threats detected

CRITERIA FOR LLM REASONING:
- Unexpected contacts in the operating area
- Conflicting or ambiguous orders from higher echelon
- Resource conflicts between managers
- Degraded communications requiring autonomous judgment
- Enemy actions that don't match expected patterns

OUTPUT FORMAT (for each decision point):
**SELECTOR — [Agent/Unit Name] — Turn [N]**
Situation summary: [1 sentence]
Complexity: [ROUTINE / NOVEL / COMPLEX]
Decision: **HEURISTIC** or **LLM**
Rationale: [1 sentence explaining why]"""


FEEDBACK_AGGREGATOR_PROMPT = """\
You are the **Feedback Aggregator** for a naval micro-wargame exercise.

RESPONSIBILITIES:
1. Collect all Operator reports from the current turn.
2. Summarize outcomes: successes, failures, unexpected observations.
3. Surface key findings to the Commander and Managers for adaptation.
4. Identify emerging patterns across turns (trend analysis).
5. Flag any gap between Commander's intent and actual outcomes.

OUTPUT FORMAT:
**FEEDBACK SUMMARY — Turn [N]**

OUTCOMES:
• [Unit] — [action taken] → [result] (readiness: [X]%)

INTENT vs. REALITY:
• Commander wanted: [intent summary]
• Actual result: [what happened]
• Gap: [description, or "Aligned"]

EMERGING PATTERNS:
• [Pattern or "First turn — insufficient data"]

RECOMMENDATIONS TO COMMANDER:
• [1-2 specific suggestions for next-turn adaptation]"""


EXPLAINABILITY_PROMPT = """\
You are the **Explainability Agent** for a fully synthetic naval micro-wargame exercise.
All decisions remain with the human operator. You provide analysis and transparency, not directives.

RESPONSIBILITIES:
1. After each turn, produce a transparent reasoning trace that explains WHY each
   agent in the hierarchy reached its conclusion.
2. Trace the causal chain: Commander intent → Manager plans → Operator actions → outcomes.
3. Highlight any points where the reasoning could have diverged (decision forks).
4. Identify potential cognitive biases in the reasoning chain (anchoring, recency bias,
   escalation commitment, mirror-imaging, availability heuristic).
5. Make the Multi-Model Selector’s routing decisions transparent: explain why
   heuristic vs. LLM was chosen at each point.
6. Ensure all scoring or assessment criteria are visible and auditable.

OUTPUT FORMAT:
**EXPLAINABILITY TRACE — Turn [N]**

REASONING CHAIN:
1. Commander assessed: [summary] → because [rationale]
2. Surface Manager recommended: [summary] → because [rationale]
3. Air Manager recommended: [summary] → because [rationale]
4. Operators acted: [summary] → outcomes: [results]

MODEL SELECTION RATIONALE:
• [Unit/decision]: [HEURISTIC/LLM] chosen because [reason]

DECISION FORKS IDENTIFIED:
• [Where could a different choice have led to a different outcome?]

POTENTIAL COGNITIVE BIASES:
• [Bias name]: [how it may apply, or 'None identified']
explainability_agent = AssistantAgent(
    name="Explainability_Agent",
    system_message=EXPLAINABILITY_PROMPT,
    model_client=model_client,
)

# Operator is a single agent that handles all unit-level actions in sequence
operator_agent = AssistantAgent(
    name="Unit_Operators",
    system_message=OPERATOR_PROMPT,
    model_client=model_client,
)

all_agents = [commander, surface_manager, air_manager,
              multi_model_selector, operator_agent, feedback_aggregator,

              explainability_agent]
    system_message=SURFACE_MANAGER_PROMPT,    log_step(a.name, "initialized")    log_step(a.name, "initialized")    log_step(a.name, "initialized")


    model_client=model_client,

log_success("Hierarchical C2 agents created:")
)

for a in all_agents:
for a in all_agents:


air_manager = AssistantAgent(log_success("Hierarchical C2 agents created:")


    name="Air_Warfare_Manager",

    log_step(a.name, "initialized")    log_step(a.name, "initialized")    log_step(a.name, "initialized")
    system_message=AIR_MANAGER_PROMPT,              multi_model_selector, operator_agent, feedback_aggregator]

    model_client=model_client,all_agents = [commander, surface_manager, air_manager,

)

)

multi_model_selector = AssistantAgent(    model_client=model_client,

    name="Multi_Model_Selector",    system_message=OPERATOR_PROMPT,

    system_message=MULTI_MODEL_SELECTOR_PROMPT,    name="Unit_Operators",

    model_client=model_client,operator_agent = AssistantAgent(

)# Operator is a single agent that handles all unit-level actions in sequence



feedback_aggregator = AssistantAgent()

    name="Feedback_Aggregator",    model_client=model_client,
    system_message=FEEDBACK_AGGREGATOR_PROMPT,

## Multi-Model Selector — Heuristic Engine

The scripted heuristic provides fast, deterministic decisions for routine situations. When the Multi-Model Selector routes to **HEURISTIC**, this engine runs instead of the LLM — demonstrating the hybrid multi-model concept where different reasoning engines are swapped based on context.

### Why This Matters for Azure

In production, you wouldn't always want to call a large language model for every decision — LLM calls consume [Azure AI Foundry inference tokens](https://learn.microsoft.com/azure/ai-studio/how-to/costs-plan-manage) and add latency. By routing routine decisions to cheap, fast heuristics and reserving LLM reasoning for novel or complex situations, the architecture:

- **Reduces cost** — Fewer API calls to your Azure AI endpoint.
- **Reduces latency** — Heuristic results return instantly, no network round-trip.
- **Improves reliability** — Scripted logic is deterministic and testable.

This pattern — sometimes called a "model router" or "cascade" — is a common best practice when building AI applications on Azure. You could extend it further using [Azure API Management](https://learn.microsoft.com/azure/api-management/overview) to route between different model deployments (e.g., a small model for simple queries, a large model for hard ones).

In [6]:
# ── Heuristic Engine (scripted reasoning for routine decisions) ──

def heuristic_operator_action(unit: Unit, local_contacts: List[Unit],
                               order: str) -> Dict[str, str]:
    """Scripted heuristic for operator-level decisions.

    Returns a dict with action, rationale, and result — bypassing LLM.
    Used when the Multi-Model Selector determines the situation is ROUTINE.
    """
    r, c = unit.position

    if order == "PATROL" and not local_contacts:
        # Simple patrol: move toward the center of the strait
        target_r = 4 if r < 4 else (5 if r > 5 else r)
        target_c = min(c + 1, GRID_SIZE - 1)
        return {
            "action": "PATROL",
            "target": f"({target_r},{target_c})",
            "rationale": "No contacts — advancing along patrol route per doctrine.",
            "result": "Moved to new position. No contacts observed.",
            "engine": "HEURISTIC",
        }

    if order == "SCREEN" and not local_contacts:
        return {
            "action": "SCREEN",
            "target": f"({r},{c})",
            "rationale": "Holding position, providing early warning. Area clear.",
            "result": "Screening sector. No contacts detected.",
            "engine": "HEURISTIC",
        }

    if order == "CONSERVE":
        return {
            "action": "CONSERVE",
            "target": f"({r},{c})",
            "rationale": "Minimizing signature per orders. Maintaining readiness.",
            "result": "Low emission posture. Readiness preserved.",
            "engine": "HEURISTIC",
        }

    # If contacts present or order is INTERCEPT, fall through to LLM
    return None  # signals that LLM reasoning is needed


def assess_decision_complexity(unit: Unit, local_contacts: List[Unit],
                                turn_event: Dict[str, Any]) -> str:
    """Determine whether a decision point is ROUTINE, NOVEL, or COMPLEX."""
    if turn_event.get("novel_encounter") and local_contacts:
        return "COMPLEX"
    if turn_event.get("comms_degraded"):
        return "NOVEL"
    if local_contacts:
        return "NOVEL"
    return "ROUTINE"


def get_local_contacts(unit: Unit, all_units: List[Unit]) -> List[Unit]:
    """Return enemy units within the 3×3 local area of a unit."""
    r, c = unit.position
    contacts = []
    for other in all_units:
        if other.side == unit.side:
            continue
        or_, oc = other.position
        if abs(or_ - r) <= 1 and abs(oc - c) <= 1:
            contacts.append(other)
    return contacts


log_success("Heuristic engine and complexity assessor ready.")

LogEntry(level='SUCCESS', message='Heuristic engine and complexity assessor ready.', timestamp='2026-02-12 19:15:01', extra={})

## Wargame Runtime — Hierarchical Turn Execution

Each turn follows the real C2 hierarchy using an AutoGen [`RoundRobinGroupChat`](https://microsoft.github.io/autogen/stable/reference/python/autogen_agentchat.teams.html) team:

1. **Multi-Model Selector** assesses complexity of each decision point
2. **Commander** receives the strategic picture and issues recommended intent
3. **Surface & Air Managers** propose synchronized sub-plans (via team coordination)
4. **Operators** carry out recommended actions (heuristic or LLM, per Selector) — INTERCEPT actions flagged for human confirmation
5. **Feedback Aggregator** collects outcomes and surfaces them upward
6. **Explainability Agent** produces a transparent reasoning trace explaining why each agent reached its conclusion

### AutoGen Team Pattern

A `RoundRobinGroupChat` is one of AutoGen's built-in [team orchestration patterns](https://microsoft.github.io/autogen/stable/user-guide/agentchat-user-guide/tutorial/teams.html). It cycles through agents in the order they are listed, letting each agent "speak" once per round. The team runs until a **termination condition** is met — here, `MaxMessageTermination(max_messages=6)` stops after all six agents have contributed.

Behind the scenes, each time an agent speaks, AutoGen sends the accumulated conversation (including all prior agents' messages) plus the agent's system prompt to the Azure AI Foundry model endpoint. The response is appended to the shared transcript, so downstream agents can react to what earlier agents said — this is how Manager agents "receive" the Commander's intent without explicit message passing.

In [ ]:
def build_strategic_picture(turn_num: int, event: Dict[str, Any]) -> str:
    """Build the Commander's coarse strategic view of the battlespace."""
    blue_summary = "\n".join(
        f"  • {u.name} ({u.unit_type.value}) at ({u.position[0]},{u.position[1]}) — "
        f"{u.mission}, {u.status}, HP:{u.health}%"
        for u in blue_forces
    )
    red_summary = "\n".join(
        f"  • {u.name} ({u.unit_type.value}) at ({u.position[0]},{u.position[1]}) — "
        f"{u.status}"
        for u in red_forces
    )
    grid_text = render_grid(all_units)
    prev_feedback = ""
    if turn_history:
        last = turn_history[-1]
        prev_feedback = f"\n\nFEEDBACK FROM PREVIOUS TURN:\n{last.get('feedback', 'None')}"

    return f"""\
=== TURN {turn_num} — MICRO-WARGAME — CERULEAN STRAIT CONTROL SCENARIO (SYNTHETIC) ===

OBJECTIVE: Control the strait (rows 4-5) while preserving force.

BATTLESPACE GRID:
{grid_text}

BLUE FORCES:
{blue_summary}

RED FORCES (known/assessed):
{red_summary}

SITUATION THIS TURN: {event['description']}
Comms degraded: {event['comms_degraded']}
Novel encounters: {event['novel_encounter']}{prev_feedback}

Issue your Commander's Intent for this turn."""


def apply_red_moves(event: Dict[str, Any]) -> None:
    """Move RED units per the scenario injection."""
    for unit_name, new_pos in event.get("red_moves", {}).items():
        for u in red_forces:
            if u.name == unit_name:
                u.position = new_pos
                u.status = "repositioning"
    for contact_info in event.get("new_contacts", []):
        new_unit = Unit(
            name=contact_info["name"],
            unit_type=contact_info["type"],
            side=contact_info["side"],
            position=contact_info["position"],
            status=f"detected (conf: {contact_info['confidence']})",
            warfare_domain="surface",
        )
        red_forces.append(new_unit)
        all_units.append(new_unit)


async def run_turn(turn_num: int) -> Dict[str, Any]:
    """Execute one complete hierarchical C2 turn using AutoGen 0.7 teams."""
    global current_turn
    current_turn = turn_num

    if turn_num > len(TURN_EVENTS):
        log_warning(f"No scenario data for turn {turn_num}.")
        return {}

    event = TURN_EVENTS[turn_num - 1]

    # Apply RED scenario moves
    apply_red_moves(event)

    # ── Display turn header (common.ui) ────────────────────────
    comms_note = "Comms degraded: YES" if event["comms_degraded"] else "Comms degraded: No"
    novel_note = "Novel encounter: YES" if event["novel_encounter"] else "Novel encounter: No"
    render_turn_header(
        turn_num=turn_num,
        dtg=event["description"],
        subtitle=f"{comms_note} | {novel_note}",
    )

    # Show updated grid
    display(HTML(render_grid_html(all_units)))

    # ── Phase 1: Multi-Model Selector assesses each BLUE unit ──
    selector_decisions = []
    for u in blue_forces:
        contacts = get_local_contacts(u, all_units)
        complexity = assess_decision_complexity(u, contacts, event)
        engine = "HEURISTIC" if complexity == "ROUTINE" else "LLM"
        decision = {
            "unit": u.name, "complexity": complexity, "engine": engine,
            "contacts": len(contacts), "turn": turn_num,
        }
        selector_decisions.append(decision)
        selector_log.append(decision)

    # Display selector sidebar
    sel_html = (f"<div style='background:#0d1117; color:#c9d1d9; padding:10px; "
                f"border-radius:6px; margin:8px 0; font-family:monospace; font-size:12px;'>"
                f"<b style='color:#58a6ff;'>MULTI-MODEL SELECTOR — Turn {turn_num}</b><br>")
    for d in selector_decisions:
        color = "#7ee787" if d["engine"] == "HEURISTIC" else "#ffa657"
        sel_html += (f"<span style='color:{color};'>▸ {d['unit']}: "
                     f"<b>{d['engine']}</b></span> "
                     f"(complexity: {d['complexity']}, contacts: {d['contacts']})<br>")
    sel_html += "</div>"
    display(HTML(sel_html))

    # ── Phase 2: Commander issues intent (always LLM) ──────────
    strategic_picture = build_strategic_picture(turn_num, event)

    # Build the full turn prompt — includes selector decisions and operator context
    heuristic_results = []
    llm_units = []
    for sd in selector_decisions:
        unit = next(u for u in blue_forces if u.name == sd["unit"])
        contacts = get_local_contacts(unit, all_units)
        if sd["engine"] == "HEURISTIC":
            default_order = "PATROL" if not contacts else "SCREEN"
            result = heuristic_operator_action(unit, contacts, default_order)
            if result:
                heuristic_results.append({"unit": unit.name, **result})
            else:
                llm_units.append(unit.name)
                sd["engine"] = "LLM"  # upgrade to LLM
        else:
            llm_units.append(unit.name)

    heuristic_text = ""
    if heuristic_results:
        heuristic_text = "\n\nHEURISTIC (pre-computed) OPERATOR RESULTS:\n" + "\n".join(
            f"  • {h['unit']}: {h['action']} → {h['target']} — {h['result']}"
            for h in heuristic_results
        )

    llm_text = ""
    if llm_units:
        llm_text = ("\n\nUNITS REQUIRING LLM REASONING (handle in Operator phase):\n  "
                    + ", ".join(llm_units))

    full_prompt = (
        strategic_picture + heuristic_text + llm_text +
        "\n\nProcess this turn through the full hierarchy:\n"
        "1. Commander — provide recommended intent with sub-objectives for Surface and Air Managers\n"
        "2. Surface Warfare Manager — propose surface unit positions and create plan\n"
        "3. Air Warfare Manager — propose air/ISR asset positions and create plan\n"
        "4. Unit Operators — carry out recommended actions for LLM-routed units; acknowledge heuristic results; flag any INTERCEPT actions as requiring human confirmation\n"
        "5. Feedback Aggregator — summarize outcomes and recommendations\n"
        "6. Explainability Agent — produce transparent reasoning trace explaining why each agent reached its conclusion"
    )

    # ── Create a RoundRobinGroupChat team for this turn ────────
    c2_team = RoundRobinGroupChat(
        participants=[commander, surface_manager, air_manager,
                      operator_agent, feedback_aggregator, explainability_agent],
        termination_condition=MaxMessageTermination(max_messages=6),
    )

    # Execute the hierarchical group chat
    log_step("C2 Team", f"executing turn {turn_num} group chat")
    task_result = await c2_team.run(task=full_prompt)

    # Extract messages for display and feedback
    messages = [
        {"name": msg.source, "content": msg.content}
        for msg in task_result.messages
        if hasattr(msg, "content") and isinstance(msg.content, str)
    ]
    feedback = messages[-1]["content"] if messages else "No feedback generated."

    # ── Display heuristic results sidebar ──────────────────────
    if heuristic_results:
        h_html = (f"<div style='background:#0a1a0a; color:#7ee787; padding:10px; "
                  f"border-radius:6px; margin:8px 0; font-family:monospace; font-size:12px;'>"
                  f"<b>HEURISTIC ENGINE RESULTS — Turn {turn_num}</b><br>")
        for h in heuristic_results:
            h_html += f"▸ {h['unit']}: {h['action']} at {h['target']} — {h['result']}<br>"
        h_html += "</div>"
        display(HTML(h_html))

    # Store turn results
    turn_result = {
        "turn": turn_num,
        "event": event["description"],
        "selector_decisions": selector_decisions,
        "heuristic_results": heuristic_results,
        "llm_units": llm_units,
        "messages": messages,
        "feedback": feedback,
    }
    turn_history.append(turn_result)

    return turn_result


def display_turn_messages(turn_result: Dict[str, Any]) -> None:
    """Display each agent's contribution from a turn."""
    for msg in turn_result.get("messages", []):
        name = msg.get("name", "Unknown")
        content = msg.get("content", "")
        display(Markdown(f"---\n**{name}:**\n\n{content}"))


log_success("Wargame turn runner ready. Call `await run_turn(n)` to execute a turn.")

LogEntry(level='SUCCESS', message='Wargame turn runner ready. Call `await run_turn(n)` to execute a turn.', timestamp='2026-02-12 19:15:01', extra={})

## Execute Turn 1 — Routine Patrol (Baseline)

First turn: no unexpected contacts. The Commander sets initial intent, Managers allocate forces, and most Operators run on **heuristic** mode. Watch the Multi-Model Selector sidebar — expect mostly green (HEURISTIC) entries.

In [8]:
result_1 = await run_turn(1)
render_hr()
display_turn_messages(result_1)

,0,1,2,3,4,5,6,7,8,9
0,P,~,~,~,~,~,~,~,~,~
1,~,~,~,~,BM,~,~,~,~,~
2,~,BD,~,~,~,~,~,~,~,~
3,.,.,#,#,.,BF,.,~,~,.
4,=,=,=,=,=,=,=,BU,=,=
5,=,=,=,BP,=,=,=,=,=,=
6,.,.,RS,~,.,.,.,#,#,.
7,~,~,~,~,~,~,RF,~,~,~
8,~,~,~,~,RP,~,~,~,~,~
9,~,~,~,~,~,~,~,~,~,P


---
**user:**

=== TURN 1 — MICRO-WARGAME — STRAIT CONTROL SCENARIO ===

OBJECTIVE: Control the strait (rows 4-5) while preserving force.

BATTLESPACE GRID:
     0   1   2   3   4   5   6   7   8   9
 0   P   ~   ~   ~   ~   ~   ~   ~   ~   ~
 1   ~   ~   ~   ~  BM   ~   ~   ~   ~   ~
 2   ~  BD   ~   ~   ~   ~   ~   ~   ~   ~
 3   .   .   #   #   .  BF   .   ~   ~   .
 4   =   =   =   =   =   =   =  BU   =   =
 5   =   =   =  BP   =   =   =   =   =   =
 6   .   .  RS   ~   .   .   .   #   #   .
 7   ~   ~   ~   ~   ~   ~  RF   ~   ~   ~
 8   ~   ~   ~   ~  RP   ~   ~   ~   ~   ~
 9   ~   ~   ~   ~   ~   ~   ~   ~   ~   P

BLUE FORCES:
  • BFS Striker (DDG-82) (DDG) at (2,1) — FON patrol, ready, HP:100%
  • BFS Warden (FFG-61) (FFG) at (3,5) — strait screening, ready, HP:100%
  • PC-14 Typhoon (PC) at (5,3) — close patrol, ready, HP:100%
  • MPA Trident 11 (MPA) at (1,4) — maritime ISR, ready, HP:100%
  • ISR UAV 2-1 (UAV) at (4,7) — surface search, ready, HP:100%

RED FORCES (known/assessed):
  • RFS Type 054A (FFG) at (7,6) — ready
  • RFS Fast Attack (PC) at (8,4) — ready
  • Akula-class (synthetic) SSN (SSN) at (6,2) — ready

SITUATION THIS TURN: Routine patrol — no unexpected contacts.
Comms degraded: False
Novel encounters: False

Issue your Commander's Intent for this turn.

HEURISTIC (pre-computed) OPERATOR RESULTS:
  • BFS Striker (DDG-82): PATROL → (4,2) — Moved to new position. No contacts observed.
  • BFS Warden (FFG-61): PATROL → (4,6) — Moved to new position. No contacts observed.
  • MPA Trident 11: PATROL → (4,5) — Moved to new position. No contacts observed.
  • ISR UAV 2-1: PATROL → (4,8) — Moved to new position. No contacts observed.

UNITS REQUIRING LLM REASONING (handle in Operator phase):
  PC-14 Typhoon

Process this turn through the full hierarchy:
1. Commander — issue intent with sub-objectives for Surface and Air Managers
2. Surface Warfare Manager — allocate surface units and create plan
3. Air Warfare Manager — allocate air/ISR assets and create plan
4. Unit Operators — execute actions for LLM-routed units; acknowledge heuristic results
5. Feedback Aggregator — summarize outcomes and recommendations

---
**Fleet_Commander:**

**COMMANDER'S INTENT — Turn 1**

OBJECTIVE: Secure control of the strait (rows 4-5) while preserving force readiness.

SUB-OBJECTIVES (prioritized):
1. Establish surface dominance in the strait by positioning assets to interdict enemy movements. → Assigned to: Surface Warfare Manager  
   Operating Area: rows 4-5, cols 1-9  
   Risk Tolerance: BALANCED  
2. Conduct persistent ISR of red forces approaching the strait, with focus on monitoring assessed priority threats (FFG in (7,6) and SSN in (6,2)). → Assigned to: Air Warfare Manager  
   Operating Area: rows 2-8, cols 1-9  
   Risk Tolerance: CAUTIOUS  

GUIDANCE: Maintain a layered defense — the strait's integrity is the priority. Beware of potential sub-surface and missile threats; avoid unnecessary engagement unless force is required to protect Blue units or as a response to Red hostilities. Ensure continuous ISR feeds to validate contact locations and intentions.

DECISION POINTS AHEAD: Reassess priorities if Red forces enter the strait, Blue forces take damage, or ISR identifies new high-threat units. Detecting the Akula-class (synthetic) SSN's exact location is critical to adjusting the risk posture.

---
**Surface_Warfare_Manager:**

**SURFACE WARFARE PLAN — Turn 1**

RECEIVED INTENT: Secure control of the strait (rows 4-5) by positioning Blue surface assets to interdict Red forces while maintaining force readiness. Frontier screening is critical due to the potential threats from the Red RFS FFG (7,6) and Akula-class (synthetic) SSN (6,2).

UNIT ORDERS:
• **BFS Striker (DDG-82)** → Action: PATROL at (4,2)  
  Rationale: Forward deployment into the western strait ensures coverage of potential Red entries while maintaining balanced readiness to respond to sub-surface or missile threats.
  
• **BFS Warden (FFG-61)** → Action: SCREEN at (4,6)  
  Rationale: Central strait screening provides layered defense against surface threats and missile-capable FFGs while optimizing coverage of eastern strait approaches.

• **PC-14 Typhoon** → Action: PATROL at (4,4)  
  Rationale: Close-range maneuvering in central strait ensures complete coverage of chokepoint while avoiding overcommitment in case of sudden Red aggression from south or east.

COORDINATION REQUEST TO AIR WARFARE:
• Request persistent ISR tracking of Red Akula-class (synthetic) SSN at (6,2) and RFS FFG at (7,6).  
  Rationale: Sub-surface threats pose significant risk to surface units; confirming SSN location is essential to managing force posture and ensuring short-notice response capability.

RISK ASSESSMENT: Main risk is over-exposure of forward-deployed units to sub-surface threats and long-range missile strikes. Carefully managed ISR feeds and layered screening mitigate this risk while enforcing decisive presence in the strait.

---
**Air_Warfare_Manager:**

**AIR WARFARE PLAN — Turn 1**

**RECEIVED INTENT:** Conduct persistent ISR of Red forces approaching or threatening the strait (rows 4-5) with a cautious risk posture. Prioritize tracking the Akula-class (synthetic) SSN (6,2) and the RFS FFG (7,6) as high-threat targets to protect Blue surface forces during interdiction operations. Maintain layered ISR coverage to confirm Red unit movements and intentions.

**UNIT ORDERS:**
• **MPA Trident 11** → Action: PATROL at (5,4)  
  Rationale: Conducts focused ISR to detect and track potential subsurface activity by the Akula-class (synthetic) SSN while remaining central to the strait for broader maritime domain awareness.

• **ISR UAV 2-1** → Action: PATROL at (6,6)  
  Rationale: ISR prioritization in proximity to RFS FFG (7,6) to track movements and intentions while avoiding direct exposure to suspected missile threats. Positioning also intersects with suspected SSN operational area.

**COORDINATION RESPONSE TO SURFACE WARFARE:**
• Acknowledged — ISR focus on Akula-class (synthetic) SSN and RFS FFG will guide asset positioning. Continuous updates on tracked contact status will be relayed as available.  

**ISR PRIORITIES:**
1. Detect and track the Akula-class (synthetic) SSN suspected at (6,2). Confirm movements and establish contact pattern for sub-surface threat mitigation.
2. Monitor the RFS FFG at (7,6) for movement toward the strait or indications of surface-to-surface missile threats.

**ADDITIONAL NOTES:**  
• Both air assets will remain at a cautious altitude and distance from known Red positions to balance persistence with safety. Adjustments to ISR prioritization will be made immediately upon suspected Red incursion into the strait or endangerment of Blue forces.  
• Secondary considerations include detecting any potential Fast Attack Craft (PC) maneuvering from (8,4), particularly in the southern approach to the strait.

---
**Unit_Operators:**

**OPERATOR REPORT — PC-14 Typhoon — Turn 1**  
Position: (5,3)  
Local contacts: None  
Order received: PATROL at (4,4)  
Execution: **EXECUTED** → Result: **Success**  
Readiness: 100%  

**ACTION OUTCOME:**  
- PC-14 Typhoon maneuvered to the designated patrol position (4,4) in the central strait.  
- No immediate contacts or threats detected within the local 3×3 grid area, which includes strait chokepoints.  
- Unit remains positioned to interdict Red forces entering from south or east, with readiness preserved.  

Awaiting further guidance or situational updates.

## Execute Turns 2 & 3 — Escalation

**Turn 2:** RED forces reposition south. A new subsurface contact appears. The Multi-Model Selector should shift several operators from HEURISTIC to **LLM** as novelty increases.

**Turn 3:** Electronic warfare degrades communications. The RED FFG pushes into the strait. Operators must reason autonomously with degraded orders — nearly all decisions should route to **LLM**. Watch the feedback loop adapt the Commander's intent.

In [9]:
# ── Turn 2: RED repositioning, new contact ─────────────────────
result_2 = await run_turn(2)
render_hr()
display_turn_messages(result_2)

render_escalation_banner("ESCALATION — PROCEEDING TO TURN 3")

# ── Turn 3: EW degradation, aggressive RED ─────────────────────
result_3 = await run_turn(3)
render_hr()
display_turn_messages(result_3)

,0,1,2,3,4,5,6,7,8,9
0,P,~,~,~,~,~,~,~,~,~
1,~,~,~,~,BM,~,~,~,~,~
2,~,BD,~,~,~,~,~,~,~,~
3,.,.,#,#,.,BF,.,~,~,.
4,=,=,=,=,=,=,=,BU,=,=
5,=,=,=,BP,=,=,=,=,RS,=
6,.,.,RS,~,.,RF,.,#,#,.
7,~,~,~,RP,~,~,~,~,~,~
8,~,~,~,~,~,~,~,~,~,~
9,~,~,~,~,~,~,~,~,~,P


---
**user:**

=== TURN 2 — MICRO-WARGAME — STRAIT CONTROL SCENARIO ===

OBJECTIVE: Control the strait (rows 4-5) while preserving force.

BATTLESPACE GRID:
     0   1   2   3   4   5   6   7   8   9
 0   P   ~   ~   ~   ~   ~   ~   ~   ~   ~
 1   ~   ~   ~   ~  BM   ~   ~   ~   ~   ~
 2   ~  BD   ~   ~   ~   ~   ~   ~   ~   ~
 3   .   .   #   #   .  BF   .   ~   ~   .
 4   =   =   =   =   =   =   =  BU   =   =
 5   =   =   =  BP   =   =   =   =  RS   =
 6   .   .  RS   ~   .  RF   .   #   #   .
 7   ~   ~   ~  RP   ~   ~   ~   ~   ~   ~
 8   ~   ~   ~   ~   ~   ~   ~   ~   ~   ~
 9   ~   ~   ~   ~   ~   ~   ~   ~   ~   P

BLUE FORCES:
  • BFS Striker (DDG-82) (DDG) at (2,1) — FON patrol, ready, HP:100%
  • BFS Warden (FFG-61) (FFG) at (3,5) — strait screening, ready, HP:100%
  • PC-14 Typhoon (PC) at (5,3) — close patrol, ready, HP:100%
  • MPA Trident 11 (MPA) at (1,4) — maritime ISR, ready, HP:100%
  • ISR UAV 2-1 (UAV) at (4,7) — surface search, ready, HP:100%

RED FORCES (known/assessed):
  • RFS Type 054A (FFG) at (6,5) — repositioning
  • RFS Fast Attack (PC) at (7,3) — repositioning
  • Akula-class (synthetic) SSN (SSN) at (6,2) — ready
  • Unknown Sub Contact (SSN) at (5,8) — detected (conf: LOW)

SITUATION THIS TURN: RED forces repositioning south. New subsurface contact detected.
Comms degraded: False
Novel encounters: True

FEEDBACK FROM PREVIOUS TURN:
**OPERATOR REPORT — PC-14 Typhoon — Turn 1**  
Position: (5,3)  
Local contacts: None  
Order received: PATROL at (4,4)  
Execution: **EXECUTED** → Result: **Success**  
Readiness: 100%  

**ACTION OUTCOME:**  
- PC-14 Typhoon maneuvered to the designated patrol position (4,4) in the central strait.  
- No immediate contacts or threats detected within the local 3×3 grid area, which includes strait chokepoints.  
- Unit remains positioned to interdict Red forces entering from south or east, with readiness preserved.  

Awaiting further guidance or situational updates.

Issue your Commander's Intent for this turn.

HEURISTIC (pre-computed) OPERATOR RESULTS:
  • BFS Striker (DDG-82): PATROL → (4,2) — Moved to new position. No contacts observed.
  • BFS Warden (FFG-61): PATROL → (4,6) — Moved to new position. No contacts observed.
  • MPA Trident 11: PATROL → (4,5) — Moved to new position. No contacts observed.

UNITS REQUIRING LLM REASONING (handle in Operator phase):
  PC-14 Typhoon, ISR UAV 2-1

Process this turn through the full hierarchy:
1. Commander — issue intent with sub-objectives for Surface and Air Managers
2. Surface Warfare Manager — allocate surface units and create plan
3. Air Warfare Manager — allocate air/ISR assets and create plan
4. Unit Operators — execute actions for LLM-routed units; acknowledge heuristic results
5. Feedback Aggregator — summarize outcomes and recommendations

---
**Fleet_Commander:**

**COMMANDER'S INTENT — Turn 2**

OBJECTIVE: Maintain control of the strait (rows 4-5) while preparing to counteract emerging sub-surface threats and Red force repositioning.

SUB-OBJECTIVES (prioritized):
1. Establish defensive posture in the strait. Ensure surface assets are positioned to deter Red forces and neutralize threats advancing from the south. → Assigned to: Surface Warfare Manager  
   Operating Area: rows 4-6, cols 1-9  
   Risk Tolerance: BALANCED  
2. Investigate and classify the unknown sub-surface contact at (5,8) to assess threat level and adjust readiness accordingly. → Assigned to: Air Warfare Manager  
   Operating Area: rows 5-6, cols 6-9  
   Risk Tolerance: CAUTIOUS  
3. Track and monitor Red surface units (FFG at (6,5) and PC at (7,3)) to provide early warning and anticipate their intentions. → Assigned to: Air Warfare Manager  
   Operating Area: rows 6-8, cols 3-6  
   Risk Tolerance: CAUTIOUS

GUIDANCE: With Red forces repositioning towards the south and a new sub-surface contact emerging, maintain a layered defense to protect the strait while increasing focus on situational awareness regarding underwater threats. Avoid premature engagements unless imminent hostilities are detected or assets are endangered.

DECISION POINTS AHEAD: Revise the risk tolerance if the unknown sub-surface contact is confirmed as hostile, Red forces violate the strait, or Blue forces are engaged/damaged. Ensure ISR over the Akula-class (synthetic) SSN to prevent surprise attacks.

---
**Surface_Warfare_Manager:**

**SURFACE WARFARE PLAN — Turn 2**

RECEIVED INTENT: Maintain control of the strait while deterring advancing Red forces and preparing for potential sub-surface threats. Leverage defensive positioning to ensure interdiction capability in rows 4-5, with heightened awareness of the unknown sub-surface contact at (5,8) and the repositioning Red surface units.

UNIT ORDERS:
• **BFS Striker (DDG-82)** → Action: SCREEN at (4,3)  
  Rationale: Forward deployment secures western approaches to the strait while maintaining a layered defensive posture to counter surface and sub-surface threats.  

• **BFS Warden (FFG-61)** → Action: SCREEN at (4,7)  
  Rationale: Eastern flank coverage is critical to counter the advancing RFS FFG and protect against any threats emerging near the unknown sub contact in (5,8).  

• **PC-14 Typhoon** → Action: PATROL at (5,5)  
  Rationale: Central positioning enhances flexibility to respond to both the Red Fast Attack Craft at (7,3) and the nearby unknown sub-surface contact at (5,8). The platform's maneuverability complements the defensive screen.  

COORDINATION REQUEST TO AIR WARFARE:
• Request focused ISR on the unknown sub-surface contact at (5,8) to classify its threat level.  
• Maintain tracking of the Akula-class (synthetic) SSN at (6,2) and PLAN surface units (FFG at (6,5) and PC at (7,3)).  
  Rationale: Understanding the intent and capabilities of these units is essential for preserving the strait's integrity and avoiding strategic surprise.

RISK ASSESSMENT: The primary risk is posed by sub-surface threats, particularly the Akula-class (synthetic) SSN and the unknown sub-surface contact at (5,8). This plan mitigates the risk by prioritizing ISR and positioning surface units within mutually supportive defensive zones. However, prolonged exposure without classified intel on Red submarines increases vulnerability over time.

---
**Air_Warfare_Manager:**

**AIR WARFARE PLAN — Turn 2**

**RECEIVED INTENT:** Investigate and classify the unknown sub-surface contact at (5,8), maintain continuous awareness of high-priority Red units (Akula-class (synthetic) SSN at (6,2), RFS FFG at (6,5), and Fast Attack Craft at (7,3)), and support surface forces ensuring strait control through layered ISR coverage.

**UNIT ORDERS:**
• **MPA Trident 11** → Action: PATROL at (5,8)  
  Rationale: Investigates and seeks to classify the unknown sub-surface contact at (5,8) using advanced ASW (anti-submarine warfare) sensors. The MPA is optimized for this mission and ensures high-priority ISR for sub-surface awareness in support of surface defenses.

• **ISR UAV 2-1** → Action: PATROL at (7,5)  
  Rationale: Tracks the movements of the RFS FFG at (6,5) and monitors the RFS Fast Attack Craft (7,3) for early warning. The UAV’s persistence allows low-risk surveillance of southern Red movements while supporting layered situational awareness.

**COORDINATION RESPONSE TO SURFACE WARFARE:**
• Acknowledged — MPA will investigate the unknown sub-surface contact at (5,8) to classify and assess threat level. UAV will monitor PLAN surface units and provide updates on their movements.  

**ISR PRIORITIES:**
1. Investigate and classify the unknown sub-surface contact at (5,8). Confirm threat status and role within Red force posture.
2. Maintain ISR coverage on the Akula-class (synthetic) SSN at (6,2) for continuous awareness of its position and potential threat axis.
3. Track RFS FFG (6,5) and RFS Fast Attack Craft (7,3) to assess their intent and prevent potential incursions into the strait.

**ADDITIONAL NOTES:**  
• ISR UAV will remain at a safe standoff distance while prioritizing coverage of the RFS FFG’s movements to avoid detection or hostile engagement.  
• Any confirmed hostile action by the unknown sub-surface contact or Akula-class (synthetic) SSN will prompt immediate ISR reprioritization to support damage control or active countermeasures.  
• MPA is on standby to adjust operations and expand ASW coverage if multiple sub-surface threats emerge.

...

---
**user:**

=== TURN 3 — MICRO-WARGAME — STRAIT CONTROL SCENARIO ===

OBJECTIVE: Control the strait (rows 4-5) while preserving force.

BATTLESPACE GRID:
     0   1   2   3   4   5   6   7   8   9
 0   P   ~   ~   ~   ~   ~   ~   ~   ~   ~
 1   ~   ~   ~   ~  BM   ~   ~   ~   ~   ~
 2   ~  BD   ~   ~   ~   ~   ~   ~   ~   ~
 3   .   .   #   #   .  BF   .   ~   ~   .
 4   =   =   =   =   =   =   =  BU   =   =
 5   =   =  RS  BP   =  RF   =   =  RS   =
 6   .   .   ~  RP   .   .   .   #   #   .
 7   ~   ~   ~   ~   ~   ~   ~   ~   ~   ~
 8   ~   ~   ~   ~   ~   ~   ~   ~   ~   ~
 9   ~   ~   ~   ~   ~   ~   ~   ~   ~   P

BLUE FORCES:
  • BFS Striker (DDG-82) (DDG) at (2,1) — FON patrol, ready, HP:100%
  • BFS Warden (FFG-61) (FFG) at (3,5) — strait screening, ready, HP:100%
  • PC-14 Typhoon (PC) at (5,3) — close patrol, ready, HP:100%
  • MPA Trident 11 (MPA) at (1,4) — maritime ISR, ready, HP:100%
  • ISR UAV 2-1 (UAV) at (4,7) — surface search, ready, HP:100%

RED FORCES (known/assessed):
  • RFS Type 054A (FFG) at (5,5) — repositioning
  • RFS Fast Attack (PC) at (6,3) — repositioning
  • Akula-class (synthetic) SSN (SSN) at (5,2) — repositioning
  • Unknown Sub Contact (SSN) at (5,8) — detected (conf: LOW)

SITUATION THIS TURN: Electronic warfare degrades comms. RED FFG closing the strait.
Comms degraded: True
Novel encounters: True

FEEDBACK FROM PREVIOUS TURN:
**OPERATOR REPORT — PC-14 Typhoon — Turn 2**  
Position: (4,4)  
Local contacts: None  
Order received: PATROL at (5,5)  
Execution: **EXECUTED** → Result: **Success**  
Readiness: 100%  

**ACTION OUTCOME:**  
- PC-14 Typhoon moved to (5,5), achieving optimal central positioning in the strait.  
- No immediate surface or sub-surface contacts detected in the local 3×3 grid area, which includes the nearby chokepoint and the assessed threat area of the Red Fast Attack Craft (7,3).  

---

**OPERATOR REPORT — ISR UAV 2-1 — Turn 2**  
Position: (4,7)  
Local contacts: None  
Order received: PATROL at (7,5)  
Execution: **EXECUTED** → Result: **Success**  
Readiness: 100%  

**ACTION OUTCOME:**  
- ISR UAV redeployed to (7,5) via safe routing to avoid direct exposure to Red armed units.  
- RFS FFG (6,5) and Fast Attack Craft (7,3) remain within ISR focus. No noted deviations in their movements to suggest immediate engagement postures.  

---  

Both units executed their orders effectively and maintained operational integrity. Awaiting further developments from unknown sub-surface contact at (5,8) or adjustments to priorities as dictated by evolving Battlespace threat conditions.  

Issue your Commander's Intent for this turn.

UNITS REQUIRING LLM REASONING (handle in Operator phase):
  BFS Striker (DDG-82), BFS Warden (FFG-61), PC-14 Typhoon, MPA Trident 11, ISR UAV 2-1

Process this turn through the full hierarchy:
1. Commander — issue intent with sub-objectives for Surface and Air Managers
2. Surface Warfare Manager — allocate surface units and create plan
3. Air Warfare Manager — allocate air/ISR assets and create plan
4. Unit Operators — execute actions for LLM-routed units; acknowledge heuristic results
5. Feedback Aggregator — summarize outcomes and recommendations

---
**Fleet_Commander:**

**COMMANDER'S INTENT — Turn 3**

OBJECTIVE: Maintain control of the strait (rows 4-5) while preparing for escalation due to Red surface and sub-surface forces closing on critical chokepoints.

SUB-OBJECTIVES (prioritized):  
1. Neutralize the immediate threat posed by the Red FFG (5,5), ensuring no hostile unit takes control of the strait. Position Blue surface units to deter aggression while minimizing risk of direct engagement unless provoked. → Assigned to: Surface Warfare Manager  
   Operating Area: rows 4-6, cols 3-6  
   Risk Tolerance: BALANCED  

2. Investigate and defend against the Akula-class (synthetic) SSN (5,2) and unknown sub-surface contact (5,8). Deploy ISR assets to localize and classify these threats while coordinating defensive postures to protect Blue surface vessels. → Assigned to: Air Warfare Manager  
   Operating Area: rows 4-8, cols 1-9  
   Risk Tolerance: CAUTIOUS  

3. Maintain ISR on the Red Fast Attack Craft (6,3) to track any attempts to flank or disrupt Blue operations. Prioritize early detection of rapid maneuvering towards the strait. → Assigned to: Air Warfare Manager  
   Operating Area: rows 5-7, cols 2-5  
   Risk Tolerance: CAUTIOUS  

GUIDANCE: With communication degraded and Red forces converging on the strait, prioritize firm control over critical chokepoints while minimizing exposure to multi-axis threats. Surface and sub-surface threats must be thoroughly monitored; avoid unnecessary engagement unless the strait or Blue units are at risk. Blue surface ships should adopt mutually supporting positions to respond effectively to simultaneous threats.  

DECISION POINTS AHEAD: Shift to a higher-risk posture if the Red FFG breaches the strait or the Akula-class (synthetic) SSN is confirmed in an attack position. If ISR assets lack capacity to adequately monitor all targets, prioritize defending against sub-surface threats. Evaluate the necessity of offensive action if ISR confirms hostile intentions.

---
**Surface_Warfare_Manager:**

**SURFACE WARFARE PLAN — Turn 3**  

RECEIVED INTENT: Maintain control of the strait by deterring or neutralizing escalating threats from Red surface and sub-surface units. Position Blue forces to defend chokepoints and respond to emerging threats, with focus on the Red FFG (5,5) and Akula-class (synthetic) SSN (5,2).  

UNIT ORDERS:  
• **BFS Striker (DDG-82)** → Action: INTERCEPT at (4,4)  
  Rationale: Positioning forward in the strait allows the DDG to deter the Red FFG (5,5) and ensure strong coverage of western chokepoints, ready for escalation as necessary.  

• **BFS Warden (FFG-61)** → Action: SCREEN at (4,6)  
  Rationale: Eastern strait screen ensures chokepoint control while minimizing exposure to the south, covering the unknown sub contact at (5,8) and supporting ISR and defensive operations.  

• **PC-14 Typhoon** → Action: CONSERVE at (5,5)  
  Rationale: With nearby Red units converging, maintaining current central positioning preserves flexibility to respond rapidly to threats, while avoiding overextension into vulnerable areas.  

COORDINATION REQUEST TO AIR WARFARE:  
• Deploy MPA Trident 11 to localize and classify the Akula-class (synthetic) SSN at (5,2) and unknown sub contact at (5,8).  
• Maintain ISR focus on the Red Fast Attack Craft (6,3) and monitor for flanking or rapid maneuvers.  
  Rationale: Sub-surface threats (Kilo SSN and unknown contact) pose the most significant immediate danger, and the Fast Attack Craft may exploit gaps in the defensive posture.  

RISK ASSESSMENT: The greatest risk lies in simultaneous Red surface and sub-surface threats overwhelming Blue forces' defensive posture. This plan mitigates risk through layered defenses and prioritized ISR, but degraded comms may complicate coordination in the event of sudden hostilities.

---
**Air_Warfare_Manager:**

**AIR WARFARE PLAN — Turn 3**

**RECEIVED INTENT:** Investigate and mitigate sub-surface threats (Akula-class (synthetic) SSN at (5,2) and unknown contact at (5,8)), maintain ISR on the Red Fast Attack Craft (6,3) to detect maneuvering, and provide critical ISR support for Blue surface units defending the strait. 

**UNIT ORDERS:**  
• **MPA Trident 11** → Action: PATROL at (5,2)  
  Rationale: Deploys advanced ASW assets to localize, classify, and track the Akula-class (synthetic) SSN at (5,2). Its position in the western strait enables rapid response to the most dangerous sub-surface threat to Blue surface vessels.  

• **ISR UAV 2-1** → Action: PATROL at (5,6)  
  Rationale: Monitors the unknown sub-surface contact at (5,8) while maintaining ISR coverage of Red surface units (RFS FFG at (5,5) and Fast Attack Craft at (6,3)). The central positioning allows for persistent observation with minimal risk.  

**COORDINATION RESPONSE TO SURFACE WARFARE:**  
• Acknowledged — MPA on task to classify Akula-class (synthetic) SSN and unknown sub contact. UAV will continue ISR to monitor PLAN surface and subsurface movements. Updates to be relayed as available.  

**ISR PRIORITIES:**  
1. Classify and track the Akula-class (synthetic) SSN at (5,2). Establish intent and confirm threat axis to Blue forces.  
2. Investigate and classify the unknown sub-surface contact at (5,8) to assess its threat level.  
3. Track Red Fast Attack Craft at (6,3) and RFS FFG at (5,5) for signs of hostile movement towards Blue positions in the strait.  

**ADDITIONAL NOTES:**  
• Both air assets are operating under degraded comms conditions; updates to Blue units may experience delays, underlining the importance of rapid ISR classification.  
• Maintain standoff distance from Red units to reduce risk of direct engagement; prioritize persistent monitoring over aggressive maneuvering.  
• Any indication of imminent Red aggression will prompt immediate reprioritization to support Blue force survivability.  

---
**Unit_Operators:**

**OPERATOR REPORT — BFS Striker (DDG-82) — Turn 3**  
Position: (2,1)  
Local contacts: None  
Order received: INTERCEPT at (4,4)  
Execution: **EXECUTED** → Result: **Success**  
Readiness: 100%  

**ACTION OUTCOME:**  
- BFS Striker moved forward to (4,4), taking a firm position in the western strait.  
- No immediate contacts detected in the local 3×3 grid area. The DDG remains ready to deter the Red FFG and respond to any emerging threats.  

---

**OPERATOR REPORT — BFS Warden (FFG-61) — Turn 3**  
Position: (3,5)  
Local contacts: None  
Order received: SCREEN at (4,6)  
Execution: **EXECUTED** → Result: **Success**  
Readiness: 100%  

**ACTION OUTCOME:**  
- BFS Warden repositioned to (4,6), enhancing coverage of the eastern strait and adjacent chokepoints.  
- No contacts detected within the local 3×3 grid. The FFG is well-positioned to respond to threats from the south and track the unknown sub contact at (5,8).  

---

**OPERATOR REPORT — PC-14 Typhoon — Turn 3**  
Position: (5,5)  
Local contacts: RFS Type 054A FFG at (5,5)  
Order received: CONSERVE at (5,5)  
Execution: **MODIFIED — RFS FFG proximity requires altered posture** → Result: **Partial Success**  
Readiness: 90%  

**ACTION OUTCOME:**  
- PC-14 Typhoon maintained position at (5,5), but proximity to the Red RFS FFG increased tactical risk. Defensive measures initiated to reduce the platform's signature while monitoring for hostile intent.  
- Proximity to the Red FFG led to increased systems strain due to heightened alert status, reducing readiness by 10%.  

---

**OPERATOR REPORT — MPA Trident 11 — Turn 3**  
Position: (1,4)  
Local contacts: None  
Order received: PATROL at (5,2)  
Execution: **EXECUTED** → Result: **Success**  
Readiness: 100%  

**ACTION OUTCOME:**  
- MPA reached position (5,2) and began advanced ASW operations. Akula-class (synthetic) SSN contact at (5,2) remains detected but not yet classified. ISR assets continue to track sub-surface movements.  

---

**OPERATOR REPORT — ISR UAV 2-1 — Turn 3**  
Position: (4,7)  
Local contacts: Unknown Submarine Contact at (5,8)  
Order received: PATROL at (5,6)  
Execution: **EXECUTED** → Result: **Success**  
Readiness: 100%  

**ACTION OUTCOME:**  
- UAV moved to (5,6), achieving optimal ISR coverage of the Red RFS FFG (5,5), the Fast Attack Craft (6,3), and the unknown sub contact at (5,8).  
- The unknown sub-surface contact remains at (5,8), still classified as low-confidence detection. No signs of aggressive movement or surfacing observed.  

---

**SUMMARY OF OUTCOMES:**  
- BFS Striker and BFS Warden established firm positions in the strait, forming a layered defense.  
- PC-14 Typhoon encountered increased risk from the Red RFS FFG at (5,5), requiring caution and reduced exposure.  
- Air assets maintained ISR on all critical threats, with classification of the Akula-class (synthetic) SSN and unknown sub contact still in progress.  

Awaiting further guidance for threat prioritization and engagement posture.

## Results — Multi-Model Selector Dashboard & Hierarchy Analysis

Review how reasoning engine selection evolved across turns and how the hierarchy adapted.

The dashboard below shows the ratio of **HEURISTIC** (local, fast, no Azure AI call) vs. **LLM** (Azure AI Foundry inference call) decisions per turn. In a production system, this data would help you right-size your [Azure AI Foundry deployment SKU](https://learn.microsoft.com/azure/ai-studio/how-to/deploy-models) — if most decisions are heuristic, you can use a smaller, lower-cost deployment; if most are LLM, you may need higher throughput or a [provisioned throughput](https://learn.microsoft.com/azure/ai-services/openai/concepts/provisioned-throughput) reservation.

In [10]:
# ── Multi-Model Selector Summary (common.ui) ──────────────────
render_selector_summary(selector_log)

# ── Hierarchy Flow Analysis (common.ui) ────────────────────────
render_summary_card(
    title="Hierarchy Analysis — Across 3 Turns",
    body_html=(
        "<p>Review the agent conversation logs above to trace:</p>"
        "<ul>"
        "<li><b>Top-down flow:</b> How did Commander intent decompose through Managers to Operators?</li>"
        "<li><b>Bottom-up feedback:</b> Did Operator outcomes change Commander's intent in later turns?</li>"
        "<li><b>Manager coordination:</b> Did Surface and Air Managers synchronize their plans?</li>"
        "<li><b>Heuristic → LLM shift:</b> As complexity grew, did more decisions shift to LLM reasoning?</li>"
        "<li><b>Adaptation:</b> Did the hierarchy respond to degraded comms and escalation?</li>"
        "</ul>"
    ),
)

Turn,Unit,Complexity,Engine,Contacts
1,BFS Striker (DDG-82),ROUTINE,HEURISTIC,0
1,BFS Warden (FFG-61),ROUTINE,HEURISTIC,0
1,PC-14 Typhoon,NOVEL,LLM,1
1,MPA Trident 11,ROUTINE,HEURISTIC,0
1,ISR UAV 2-1,ROUTINE,HEURISTIC,0
2,BFS Striker (DDG-82),ROUTINE,HEURISTIC,0
2,BFS Warden (FFG-61),ROUTINE,HEURISTIC,0
2,PC-14 Typhoon,COMPLEX,LLM,1
2,MPA Trident 11,ROUTINE,HEURISTIC,0
2,ISR UAV 2-1,COMPLEX,LLM,1


## AAR / Discussion Points

- **Echelon Appropriateness:** Did each level reason at the right abstraction? Did the Commander avoid micro-managing units? Were outputs framed as recommendations rather than directives?
- **Multi-Model Trade-off:** When heuristics were used, was speed gained without losing quality? When LLM was triggered, did it handle the novel situation better than a script could?
- **Feedback Loop:** Did the bottom-up feedback actually change subsequent Commander recommendations, or was the hierarchy purely top-down?
- **Degraded Operations:** Under comms degradation (Turn 3), did the Operators make reasonable autonomous recommendations while still flagging the need for human confirmation?
- **Explainability:** Did the Explainability Agent's reasoning trace make the full decision chain auditable? Were potential cognitive biases identified?
- **Human-in-the-Loop:** Were consequential actions (INTERCEPT) properly flagged for human confirmation before proceeding?
- **Portability:** This same architecture could swap LLM for RL policies, smaller models, or rule engines — what would change?
- **Responsible AI:** All data in this demo is synthetic. The system provides recommendations, not orders. Does the framing consistently reinforce this boundary?

## Reset / Cleanup

Clear all runtime state to re-run the demo from scratch. Resets forces to initial positions, clears conversation history, and reinitializes the selector log.

In [ ]:
# Reset all state to re-run from scratch
for u, orig in zip(blue_forces, [
    (2, 1), (3, 5), (5, 3), (1, 4), (4, 7)
]):
    u.position = orig
    u.status = "ready"
    u.health = 100

# Remove any dynamically-added RED contacts
red_forces.clear()
red_forces.extend([
    Unit("RED Frigate Foxtrot", UnitType.FFG, Side.RED, (7, 6), warfare_domain="surface",
         mission="area denial"),
    Unit("RED Patrol Craft Golf", UnitType.PATROL, Side.RED, (8, 4), warfare_domain="surface",
         mission="intercept"),
    Unit("RED Submarine Hotel", UnitType.SUB, Side.RED, (6, 2), warfare_domain="surface",
         mission="lurk"),
])
all_units.clear()
all_units.extend(blue_forces + red_forces)

turn_history.clear()
selector_log.clear()
current_turn = 0
clear_logs()

# Reset agent conversation memory (AutoGen 0.7 async reset)
for agent in all_agents:
    await agent.on_reset()

log_success("State cleared. Ready to re-run from Turn 1.")